In [16]:
# Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
# in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [17]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


In [18]:
# Import data from wikipedia using pandas

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]

In [19]:
# Convert scraped data into dataframe df2

df2 = pd.DataFrame(df)
df2 = df2.rename(columns={'Postal Code': 'PostalCode'})
df2.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [20]:
# Ignores cells that are not assigned a borough
df2 = df2[~df2.Borough.str.contains("Not assigned")].reset_index(drop=True)

# Replace cells that has boroughs but not assigned neighborhood with boroughs as neighborhood
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == "Not assigned", df2['Borough'], df2['Neighbourhood'])
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [21]:
# Group by rows having same Postal Code but different neighbourhood

group = df2.groupby('PostalCode')
print("Length of the group = ", len(group), "and length of the dataframe = ", len(df2))
print("The length of the dataframe and the length of the group are same, which implies that there is no need for grouping")

Length of the group =  103 and length of the dataframe =  103
The length of the dataframe and the length of the group are same, which implies that there is no need for grouping


In [22]:
# Print the numnber of rows
print(df2.shape)

(103, 3)


In [23]:
# Second part of the assignment: Adding latitude and longitude data to the dataframe
# I have used the provided csv file "http://cocl.us/Geospatial_data" to update the location

In [24]:
# Load data from csv file
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
geo_data = pd.DataFrame(geo_data)
geo_data = geo_data.rename(columns={'Postal Code': 'PostalCode'})

geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
# Add latituide and longitude columns to df2 dataframe from geo_data to create new dataframe df2_geo

df2_geo = pd.merge(df2, geo_data, on=['PostalCode'], how='left')
df2_geo.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
# Part 3 of the assignment

In [26]:
# Segmenting and Clustering Neighborhoods in Toronto
# In this section, we replace the analysis done in case of NY data for boroughs that contain Toronto 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium==0.5.0
import folium
print('Libraries imported.')

Libraries imported.


In [31]:
#Slice data to work with only boroughs that contain the word Toronto

In [28]:
Toronto_data = df2_geo[df2_geo['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [33]:
!pip install -c conda-forge geopy --yes # 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [38]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [44]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [46]:
# Cluster Neighborhood

In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 